# ASX rate tracker

Note: Data prior to 12 April sourced from Matt Cowgil's github site:
https://github.com/MattCowgill/cash-rate-scraper.git

## Python set-up

In [1]:
import glob
from pathlib import Path
from functools import cache

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:

import plotting as pg
import rba_data_capture as rdc

In [3]:
# save charts in this notebook
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/ASX/"
pg.set_chart_dir(CHART_DIR)
pg.clear_chart_dir(CHART_DIR)

RFOOTER = 'Source: ASX'
LFOOTER = 'Australia. '
FOOTERS = {'lfooter': LFOOTER, 'rfooter': RFOOTER}

# True to see the charts in the notebook
SHOW=False

In [4]:
# Save the combined data to a file (for posterity?)
SAVE_DIR = "./ASX-COMBINED/"
Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
SAVE_FILE = SAVE_DIR + "ASX-COMBINED.csv"

## ASX data cleaning and aggregation

In [5]:
def aggregate_data() -> pd.DataFrame:
    """Aggregate daily cash rate data into a single dataframe.
    Delete daily data that looks odd."""

    # Find files 
    directory = "./ASX_DAILY_DATA/"
    file_stem = 'scraped_cash_rate_'
    pattern = f"{directory}{file_stem}*.csv"
    files = glob.glob(pattern)

    # Read each file into a dataframe and then put it in a dictionary
    dict_of_series = {}
    for file in files:
        day_data = pd.read_csv(file, index_col=0) 
        scraped_date = day_data['scrape_date'].iloc[0]
        cash_rate_day = day_data['cash_rate'].round(3)
        if cash_rate_day.isnull().any():
            # drop the date if there are any missing values
            continue
        cash_rate_day.index = pd.PeriodIndex(cash_rate_day.index, freq='M')
        dict_of_series[scraped_date] = cash_rate_day

    # Create a new dataframe, order rhw rows and columns
    combined_df = (
        pd.DataFrame(dict_of_series)
        .T
        .sort_index(ascending=True)
        .sort_index(ascending=True, axis=1)
    )
    combined_df.index = pd.PeriodIndex(combined_df.index, freq='D')

    # This list comes from Matt Cowgil's code
    matts_drop_list = [
        "2022-08-06", "2022-08-07", "2022-08-08", "2023-01-18",
        "2023-01-24", "2023-01-31", "2023-02-02", "2022-12-30",
        "2022-12-29"
    ]
    combined_df = combined_df.drop(matts_drop_list, errors='ignore')

    # drop saturday/sunday data
    combined_df = combined_df[~combined_df.index.dayofweek.isin([5, 6])]

    # save to file
    combined_df.to_csv(SAVE_FILE)

    return combined_df


df = aggregate_data()

In [6]:
def data_capture_by_month():
    ax = df.groupby([df.index.year, df.index.month]).agg({'count'}).max(axis=1).plot.bar()
    pg.finalise_plot(
        ax, 
        title='Data Capture by Month', 
        xlabel='Month',
        ylabel='Number of Data Points',
        **FOOTERS,
        show=SHOW,
    )



data_capture_by_month()

## Anticipated RBA Official Cash Rates

In [7]:
def plot_anticipated():

    data = df.T.copy()
    columns = [f"_{x}" for x in data.columns]
    data.columns = columns
    num_columns = len(data.columns)
    colors = sns.color_palette("plasma", num_columns)
    ax = data.plot(color=colors, lw=0.75, alpha=0.5)
    columns[-1] = columns[-1].replace('_', '')
    data.columns = columns
    ax = data[data.columns[-1]].plot(color='green', lw=2, label='Most recent data')

    pg.finalise_plot(
        ax, 
        title='Market Anticipated RBA Policy Rates', 
        ylabel='Policy Rate (%/year)',
        y0=True,
        zero_y=True,
        **FOOTERS,
        legend={'loc': 'best', 'fontsize': 'x-small'},
        show=SHOW,
    )


plot_anticipated()

## Monthly against RBA rate


In [8]:
@cache
def periodic_rba(freq='M') -> pd.Series:
    """Get the RBA cash rate data from the RBA website."""

    # get the data
    _a2_meta, a2_data = rdc.get_data("Monetary Policy Changes – A2")
    a2_data = a2_data['ARBAMPCNCRT']
    a2_data.index = pd.PeriodIndex(a2_data.index, freq=freq)
    drops = a2_data.index.duplicated(keep='last')
    a2_data = a2_data[~drops]

    # add today's data if it is missing
    today = pd.Period(pd.Timestamp("today"), freq=freq)
    if today > a2_data.index[-1]:
        last = a2_data.iloc[-1]
        a2_data[today] = last
        a2_data = a2_data.sort_index()

    # restore missing periods
    new_index = pd.period_range(start=a2_data.index.min(), end=a2_data.index.max())
    a2_data = a2_data.reindex(new_index, fill_value=np.nan).ffill()

    # drop data before 2022-04-01
    commencing = "2022-04-01"
    a2_data = a2_data[a2_data.index >= commencing]

    return a2_data

In [9]:
def plot_against_rba():

    # Get end-of-month ASX forecast data
    asx_data = df.copy()
    asx_data.index = pd.PeriodIndex(asx_data.index, freq='M')
    drops = asx_data.index.duplicated(keep='last')
    asx_data = asx_data[~drops].T

    # plot the ASX data
    num_columns = len(asx_data.columns)
    colors = sns.color_palette("viridis", num_columns)
    ax = asx_data.plot(color=colors, lw=0.75, alpha=0.75)

    # Get RBA data, extract
    a2_data = periodic_rba(freq='M')

    # plot the RBA data
    ax = a2_data.plot(
        ax=ax,
        color="red", 
        lw=2, 
        label="RBA Cash Rate",
        drawstyle='steps-post',
    )

    pg.finalise_plot(
        ax, 
        title='End of Month Market Anticipated RBA Policy Rates', 
        ylabel='Policy Rate (%/year)',
        y0=True,
        zero_y=True,
        **FOOTERS,
        legend={'loc': 'lower right', 'fontsize': 'xx-small', 'ncols': 4},
        show=SHOW,
    )


plot_against_rba()

## Number of 25 basis point movements anticipated.

In [10]:
rba = periodic_rba(freq='D')
floor = df.copy().T.min()
peak = df.copy().T.max()
climb = (peak - rba) / 0.25
climb = climb.dropna().astype(int).where(climb > 0, 0)
ax = climb.plot(lw=1)
pg.finalise_plot(
    ax, 
    title='25 Basis Point Increases Anticipated Over Next 18 Months', 
    ylabel='Number of Whole Increases',
    y0=True,
    zero_y=True,
    rfooter=RFOOTER,
    lfooter=f"{LFOOTER}Number of anticipated whole 25 basis point rate increases above the current RBA rate. ",
    show=SHOW,
)

In [11]:
# Anticipated 25 basis point cuts, after peak
peak = (rba + climb * 0.25).dropna()
peak = peak.where(peak > rba[peak.index], rba[peak.index])
ax = peak.plot(lw=1)
pg.finalise_plot(
    ax, 
    title='Anticipated Peak RBA Policy Rate Over Next 18 Months', 
    ylabel='Policy Rate (%/year)',
    y0=True,
    zero_y=True,
    **FOOTERS,
    show=SHOW,
)


In [12]:
# Anticipated 25 basis point cuts, after peak
endpoint = df.copy().T.ffill().iloc[-1]
cuts = (peak - endpoint) * 100
cuts = cuts.dropna().where(cuts > 0, 0)
ax = cuts.plot(lw=1)

pg.finalise_plot(
    ax, 
    title='Basis Point Cuts Anticipated Over Next 18 Months', 
    ylabel='Number of Cuts',
    y0=True,
    zero_y=True,
    rfooter=RFOOTER,
    lfooter=f"{LFOOTER} Number of anticipated basis point rate cuts below the anticipated peak RBA rate. ",
    show=SHOW,
)

In [13]:
# timing of first cut
# Note - really it is only since December 2023 that markets have
# started to consistently anticipate cuts over the forward 18 months

start = "2023-12-01"
peak2 = peak[peak.index >= start]
forecasts = df.copy().T
forecasts = forecasts[forecasts.columns[forecasts.columns >= start]]
forecasts = forecasts[forecasts.index >= start]
forecasts = forecasts.sub(peak2, axis=1)
forecasts = forecasts.where(forecasts <= -0.25, np.nan)
first = pd.Series([forecasts[col].first_valid_index().to_timestamp(how='end') for col in forecasts.columns])
first.index = forecasts.columns
ax = first.plot(lw=1)

pg.finalise_plot(
    ax, 
    title='When the market fully anticipates the first rate cut', 
    ylabel='Month of first cut',
    y0=True,
    rfooter=RFOOTER,
    lfooter=f"{LFOOTER} Month market pricing first fully anticipates a 25 basis point rate cut. ",
    show=SHOW,
)

## Finished

In [14]:
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Apr 13 2024 22:01:54

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

pandas    : 2.2.2
seaborn   : 0.13.2
matplotlib: 3.8.4
numpy     : 1.26.4

Watermark: 2.4.3

